In [1]:
from jina import Flow, Executor, requests, Document, DocumentArray
import pandas as pd
import numpy as np 

### 3 main fundamental jina concepts 

* document
* executor 
* flow

In [2]:
df = pd.read_csv('lyrics-JINA - Sheet1.csv')
df.head()

,Index,Singer,Song,Lyrics,Collected By
0,1,Bruno Mars,24 K Magic,Tonight\nI just want to take you higher\nThrow...,Ishita
1,2,Taylor Swift,Wildest Dreams,"He said, ""Let's get out of this town\nDrive ou...",Ishita
2,3,Green Day,Numb,I'm tired of being what you want me to be\nFee...,Ishita
3,4,Macklemore,Glorius,You know I'm back like I never left\nAnother s...,Ishita
4,5,Ariana Grande,POV,It's like you got superpowers\nTurn my minutes...,Ishita


In [3]:
df = df.drop_duplicates().dropna()
df.shape

(33, 5)

### sample songs

In [19]:
for ind in range(3):
    print()
    print(f" Song name : {df.iloc[ind, 2]} ".center(80, ' '))
    print(f" Singer Name : {df.iloc[ind, 1]} ".center(80, ' '))
    # print(f" Singer name : {df.iloc[ind, 0]} ".center(80, ' '))
    print()
    print(df.iloc[ind, 3])
    print()
    print('-' * 80)


                             Song name : 24 K Magic                             
                            Singer Name : Bruno Mars                            

Tonight
I just want to take you higher
Throw your hands up in the sky
Let's set this party off right
Players, put yo' pinky rings up to the moon
Girls, what y'all trying to do?
Twenty four karat magic in the air
Head to toe so player
Look out uh
Pop pop, it's show time (show time)
Show time (show time)
Guess who's back again?
Oh they don't know? (Go on tell 'em)
Oh they don't know? (Go on tell 'em)
I bet they know soon as we walk in (showin' up)
Wearing Cuban links (ya)
Designer minks (ya)
Inglewood's finest shoes (whoop, whoop)
Don't look too hard might hurt ya'self
Known to give the color red the blues
Oh shit, I'm a dangerous man with some money in my pocket (keep up)
So many pretty girls around me and they waking up the rocket (keep up)
Why you mad, fix ya face, ain't my fault y'all be jocking (keep up)
Players only, com

### creating a document and documentArray

In [5]:
# basic document 

# Document(text = 'something')  # use it for strings 
# Document(content = 'something') # use it when you dont know what type of jina data type it is 
# Document(blob = 'img, vid, audio') # use it for images, audios and videos
# Document(uri = 'local file path or url link') # can be any form a local or remote link

In [16]:
docs = DocumentArray()
for ind in range(df.shape[0]):
    song_name = df.iloc[ind, 2]
    lyr = df.iloc[ind, 3]
    doc = Document(text = song_name)
    doc.tags['Lyrics'] = lyr
    docs.append(doc)
    
docs

<jina.types.arrays.document.DocumentArray length=33 at 140628195378608>

In [17]:
docs[0].json()

'{\n  "id": "88e2e994-373c-11ec-a791-69fbdffe8014",\n  "mime_type": "text/plain",\n  "tags": {\n    "Lyrics": "Tonight\\nI just want to take you higher\\nThrow your hands up in the sky\\nLet\'s set this party off right\\nPlayers, put yo\' pinky rings up to the moon\\nGirls, what y\'all trying to do?\\nTwenty four karat magic in the air\\nHead to toe so player\\nLook out uh\\nPop pop, it\'s show time (show time)\\nShow time (show time)\\nGuess who\'s back again?\\nOh they don\'t know? (Go on tell \'em)\\nOh they don\'t know? (Go on tell \'em)\\nI bet they know soon as we walk in (showin\' up)\\nWearing Cuban links (ya)\\nDesigner minks (ya)\\nInglewood\'s finest shoes (whoop, whoop)\\nDon\'t look too hard might hurt ya\'self\\nKnown to give the color red the blues\\nOh shit, I\'m a dangerous man with some money in my pocket (keep up)\\nSo many pretty girls around me and they waking up the rocket (keep up)\\nWhy you mad, fix ya face, ain\'t my fault y\'all be jocking (keep up)\\nPlayers 

In [18]:
docs[0].text

'24 K Magic'

### creating your own executor

In [20]:
class Clean(Executor):
    @requests
    def clean(self, docs, parameters, **kwargs):
        print(parameters)
        return DocumentArray(list(filter(lambda x : len(x.text) > 0, docs)))

### Flow

In [10]:
model = "sentence-transformers/paraphrase-distilroberta-base-v1"

flow = (
    Flow()
    .add(
        name = 'clean_text',
        uses = Clean
    )
    .add(
        name="error_text_encoder",
        uses="jinahub://TransformerTorchEncoder",
        uses_with={"pretrained_model_name_or_path": model},
    )
    .add(
        name="error_text_indexer",
        uses='jinahub://SimpleIndexer',
    )
)

In [11]:
docs = docs[:30]
docs

<jina.types.arrays.document.DocumentArray length=30 at 140628196078832>

In [26]:
for i in range(3):
    print(i.text)

AttributeError: 'int' object has no attribute 'text'

### indexing 

In [27]:
!rm -rf workspace

with flow:
    flow.index(
      inputs=docs,
        docs = docs,
        parameters = {'name' : 'somethign', 'xyz' : 'fsdfsdfsa'}
  )

error_text_encoder@20738[C]:can not load the executor from /home/sanjju/.jina/hub-packages/u9pqs8eb/config.yml
error_text_encoder@20738[E]:ExecutorFailToLoad() during <class 'jina.peapods.runtimes.zmq.zed.ZEDRuntime'> initialization
 add "--quiet-error" to suppress the exception details
Traceback (most recent call last):
  File "/home/sanjju/miniconda3/envs/jina/lib/python3.8/site-packages/jina/importer.py", line 127, in _path_import
    spec.loader.exec_module(module)
  File "<frozen importlib._bootstrap_external>", line 843, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/home/sanjju/.jina/hub-packages/u9pqs8eb/transform_encoder.py", line 7, in <module>
    import torch
ModuleNotFoundError: No module named 'torch'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/sanjju/miniconda3/envs/jina/lib/python3.8/site-packages/jina/peapods/runtimes/request_handlers/data

           Flow@17937[E]:Flow is aborted due to ['error_text_encoder'] can not be started. 


RuntimeFailToStart: 

### querying 

In [28]:
query = Document(text = input('Query product : '))
with flow:
    response = flow.search(inputs = query, return_results = True)

Query product :  24 K Magic 


error_text_encoder@20988[C]:can not load the executor from /home/sanjju/.jina/hub-packages/u9pqs8eb/config.yml
error_text_encoder@20988[E]:ExecutorFailToLoad() during <class 'jina.peapods.runtimes.zmq.zed.ZEDRuntime'> initialization
 add "--quiet-error" to suppress the exception details
Traceback (most recent call last):
  File "/home/sanjju/miniconda3/envs/jina/lib/python3.8/site-packages/jina/importer.py", line 127, in _path_import
    spec.loader.exec_module(module)
  File "<frozen importlib._bootstrap_external>", line 843, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/home/sanjju/.jina/hub-packages/u9pqs8eb/transform_encoder.py", line 7, in <module>
    import torch
ModuleNotFoundError: No module named 'torch'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/sanjju/miniconda3/envs/jina/lib/python3.8/site-packages/jina/peapods/runtimes/request_handlers/data

RuntimeFailToStart: 

### finding the best match

In [16]:
matches = response[0].docs[0].matches
matches

<jina.types.arrays.match.MatchArray length=20 at 139829488721536>

In [19]:
for m in matches:
    print()
    print(f" product name : {m.text} ".center(80, ' '))
    print()
    print(m.tags['description'])
    print()
    print('-' * 80)


              product name : Scenic Nature Stationery - Pack of 48              

Professionally printed Scenic Stationery. 4 unique scenes from nature, 12 of each for a total of 48. Measure 8.5 x 11 inches. Works through all printers and copiers. Images are ghosted to be printed on. Also can be handwritten. Perfect for personalized letters, projects, and poetry. Professionally printed in USA on premium 70 lb. bright white smooth text stock.

--------------------------------------------------------------------------------

 product name : Canson 100510926 XL Mix Media Paper Pad, 98 Pound, 7 x 10 Inch, 60 Sheets 

Canson XL Mix Media Pads were developed as the answer to the demand for a heavyweight, fine texture paper with heavy sizing for wet and dry media. This paper can be used for: sketching, drawing, light washes of ink and watercolor, collage, journaling, and more. It has quickly become the most popular paper in the Canson paper family. Mix Media paper erases well and blends easi